splitting using semantice similarity for fine tuning

In [89]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import os
import re
import itertools
import spacy
import nltk
from nltk.tokenize import sent_tokenize
pd.set_option('display.max_colwidth', None)

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Viktor_Shevchuk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
law_text = """
Article 1. Definitions

The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.

The territory of the anti-terrorist operation in the territory of Ukraine, where settlements defined in the list approved by the Cabinet of Ministers of Ukraine are located and where the anti-terrorist operation was conducted, which was initiated under the Decree of the President of Ukraine “On the Decision of National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014.

Article 2. The moratorium on the fulfilment of contractual obligations and the accrual of penalties and fines on the principal amount of debt under the credit and other contractual obligations

For the duration of the anti-terrorist operation, it is prohibited to charge the citizens of Ukraine who are registered and permanently reside in or resettled in the period starting from 14 April 2014 from localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted, penalties and/or fines on the principal amount of debt under credit agreements and loan agreements dated from 14 April 2014. The same shall apply to legal entities and individual entrepreneurs who conduct (conducted) their economic activities within the territory of localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted.

Banks and other financial institutions, as well as creditors, are obliged to cancel the penalties and/or fines accrued on the principal amount of debt under credit agreements and loan agreements during the period of the anti-terrorist operation for persons specified in this Article.

Provisions of parts one and two of this Article shall not apply to the accrual of penalties and/or fines on the principal amount of debt under credit agreements and loan agreements concluded with legal entities and individual entrepreneurs located within the territory of anti-terrorist operation, except for localities from the lists provided for in part four of Article 4 of this Law, if such agreements are concluded after 1 January 2018 or to which, by agreement of the parties, changes were made after 1 January 2018 to extend the term for the fulfilment of obligations and/or reduce the amount of interest or punitive sanctions.

For temporarily displaced higher education institutions and temporarily displaced scientific institutions to fulfil credit and other contractual obligations (except for agreements on the provision of educational services) that arose before moving to the territory controlled by the Ukrainian authorities in connection with the anti-terrorist operation, a moratorium is introduced for the period of the anti-terrorist operation.

For the duration of the anti-terrorist operation, it is prohibited to charge penalties and/or fines on the principal amount of debt owed by temporarily displaced higher education institutions and temporarily displaced scientific institutions under credit and other agreements (except for agreements on the provision of educational services) that have not ceased, starting from the date of publication of the order to change their location by the central executive authority in the field of education and science.

A temporarily displaced higher educational institution or a temporarily displaced scientific institution shall not be liable for the obligations undertaken on their behalf in the territory where the state authorities temporarily do not exercise their powers, from the date of publication of the order to change their location by the central executive authority in the field of education and science.

Article 3. Moratorium on inspections by bodies and officials authorised by law to bring about state supervision (control) in the field of economic activity.

Bodies and officials authorised by law to bring about state supervision (control) in the field of economic activity during the period and within the territory of the anti-terrorist operation are temporally prohibited from conducting scheduled and unscheduled inspections of business entities performing activity within the territory of the anti-terrorist operation, except for unscheduled inspections of business entities, which are classified as high-risk business entities according to the criteria approved by the Cabinet of Ministers of Ukraine for assessing the level of risk arising from economic activity.
"""

In [7]:
abstract_text = """
Article 1 of the Law establishes the following:
the anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine "On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 "On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;
the anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine "On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 "On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine" of April 14, 2014 N405/2014 was being carried out.

During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law).

According to Article 3 of the Law, bodies and officials that are authorized by law to carry out state supervision (oversight) in the sphere of commercial activity, during the anti-terrorist operation period and on the anti-terrorist operation territory are temporarily forbidden to hold scheduled and non-scheduled inspections of business entities that operate in the anti-terrorist operation area, except for unscheduled inspections of business entities that are classified as business entities with a high risk level, according to the business risk level assessment criteria approved by the Cabinet of Ministers of Ukraine.
"""

In [24]:
def split_into_sentences(text):
    sentences = sent_tokenize(text)
    return sentences

law_sentences = split_into_sentences(law_text)
abstract_sentences = split_into_sentences(abstract_text)

def generate_consecutive_sentence_combinations(sentences):
    sentence_combinations = []

    # Add individual sentences
    sentence_combinations.extend(sentences)

    # Add combinations of consecutive sentences
    for window_size in range(2, len(sentences) + 1):
        for i in range(len(sentences) - window_size + 1):
            combined_sentences = ' '.join(sentences[i:i + window_size])
            sentence_combinations.append(combined_sentences)

    return sentence_combinations

def compute_embeddings(sentences):
    return model.encode(sentences, convert_to_tensor=True)

law_sentence_combinations = generate_consecutive_sentence_combinations(law_sentences)

law_embeddings = compute_embeddings(law_sentence_combinations)
summary_embeddings = compute_embeddings(abstract_sentences)

pairs = []

for i, summary_embedding in enumerate(summary_embeddings):
    similarities = util.pytorch_cos_sim(summary_embedding, law_embeddings)

    # Move tensor to CPU and convert to numpy array if it's on GPU
    if similarities.is_cuda:
        similarities = similarities.cpu()

    similarities_np = similarities.numpy()

    # Find the index of the best match
    best_match_index = np.argmax(similarities_np, axis=1)[0]

    pairs.append({
        "Summary Sentence": abstract_sentences[i],
        "Best Matched Law Part": law_sentence_combinations[best_match_index]
    })

pairs_df = pd.DataFrame(pairs)
pairs_df

,Summary Sentence,Best Matched Law Part
0,"\nArticle 1 of the Law establishes the following:\nthe anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;\nthe anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014 was being carried out.","Definitions\n\nThe period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine. The territory of the anti-terrorist operation in the territory of Ukraine, where settlements defined in the list approved by the Cabinet of Ministers of Ukraine are located and where the anti-terrorist operation was conducted, which was initiated under the Decree of the President of Ukraine “On the Decision of National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No."
1,"During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law).","405/2014. Article 2. The moratorium on the fulfilment of contractual obligations and the accrual of penalties and fines on the principal amount of debt under the credit and other contractual obligations\n\nFor the duration of the anti-terrorist operation, it is prohibited to charge the citizens of Ukraine who are registered and permanently reside in or resettled in the period starting from 14 April 2014 from localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted, penalties and/or fines on the principal amount of debt under credit agreements and loan agreements dated from 14 April 2014. The same shall apply to legal entities and individual entrepreneurs who conduct (conducted) their economic activities within the territory of localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted."
2,"According to Article 3 of the Law, bodies and officials that are authorized by law to carry o

In [26]:
# 2.0

In [28]:
# Compute embeddings for individual law sentences
law_embeddings = compute_embeddings(law_sentences)
summary_embeddings = compute_embeddings(abstract_sentences)

mapped_indices = []  # Store indices of mapped individual law sentences

# Iterate over summary sentences and their embeddings
for summary_embedding in summary_embeddings:
    similarities = util.pytorch_cos_sim(summary_embedding, law_embeddings)
    if similarities.is_cuda:
        similarities = similarities.cpu()
    similarities_np = similarities.numpy()
    best_match_index = np.argmax(similarities_np, axis=1)[0]
    mapped_indices.append(best_match_index)

pairs = []

# Iterate over the mapped indices and append in-between sentences
for i in range(len(mapped_indices)):
    current_index = mapped_indices[i]
    next_index = mapped_indices[i + 1] if i < len(mapped_indices) - 1 else len(law_sentences)

    # Append the in-between sentences from the original law text
    in_between_text = ' '.join(law_sentences[current_index + 1:next_index])
    full_mapped_text = law_sentences[current_index] + ' ' + in_between_text

    pairs.append({
        "Summary Sentence": abstract_sentences[i],
        "Mapped Law Part": full_mapped_text
    })

pairs_df = pd.DataFrame(pairs)
pairs_df

,Summary Sentence,Mapped Law Part
0,"\nArticle 1 of the Law establishes the following:\nthe anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;\nthe anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014 was being carried out.","The territory of the anti-terrorist operation in the territory of Ukraine, where settlements defined in the list approved by the Cabinet of Ministers of Ukraine are located and where the anti-terrorist operation was conducted, which was initiated under the Decree of the President of Ukraine “On the Decision of National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014. Article 2."
1,"During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law).","The moratorium on the fulfilment of contractual obligations and the accrual of penalties and fines on the principal amount of debt under the credit and other contractual obligations\n\nFor the duration of the anti-terrorist operation, it is prohibited to charge the citizens of Ukraine who are registered and permanently reside in or resettled in the period starting from 14 April 2014 from localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted, penalties and/or fines on the principal amount of debt under credit agreements and loan agreements dated from 14 April 2014. The same shall apply to legal entities and individual entrepreneurs who conduct (conducted) their economic activities within the territory of localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted. Banks and other financial institutions, as well as creditors, are obliged to cancel the penalties and/or fines accrued on the principal amount of debt under credit agreements and loan agreements during the period of the anti-terrorist operation for persons specified in this Article. Provisions of parts one and two of this Article shall not apply to the accrual of penalties and/or fines on the principal amount of debt under credit agreements and loan agreements concluded with legal entities and individual entrepreneurs located within the territory of anti-terrorist operation, except for localities from the lists provided for in part four of Ar

In [86]:
abstracts_path = './laws_with_abstracts/split_by_sections/en_abstracts'
laws_path = './laws_with_abstracts/split_by_sections/en_laws'

In [90]:
def numerical_sort_key(s):
    numbers = re.findall(r'\d+', s)
    return tuple(int(number) for number in numbers)

def read_text_files(path, has_sections=True):
    text_data = []
    # Filter out any files to ensure we only have directories
    law_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    # Sort directories numerically
    law_dirs = sorted(law_dirs, key=numerical_sort_key)
    for law_dir in law_dirs:
        law_path = os.path.join(path, law_dir)
        if has_sections:
            # Filter and sort section directories numerically
            section_dirs = [d for d in os.listdir(law_path) if os.path.isdir(os.path.join(law_path, d))]
            section_dirs = sorted(section_dirs, key=numerical_sort_key)
            for section_dir in section_dirs:
                section_path = os.path.join(law_path, section_dir)
                # Sort text files numerically
                files = sorted([f for f in os.listdir(section_path) if f.endswith('.txt')], key=numerical_sort_key)
                for file in files:
                    file_path = os.path.join(section_path, file)
                    with open(file_path, 'r') as f:
                        text = f.read().strip()
                        text_data.append((law_dir, section_dir, file.split('.')[0], text))
        else:
            # Sort text files numerically
            files = sorted([f for f in os.listdir(law_path) if f.endswith('.txt')], key=numerical_sort_key)
            for file in files:
                file_path = os.path.join(law_path, file)
                with open(file_path, 'r') as f:
                    text = f.read().strip()
                    text_data.append((law_dir, file.split('.')[0], text))
    return text_data

abstracts_data = read_text_files(abstracts_path, has_sections=False)
laws_data = read_text_files(laws_path, has_sections=True)

abstracts_df = pd.DataFrame(abstracts_data, columns=['abstract_index', 'para_index', 'text'])
laws_df = pd.DataFrame(laws_data, columns=['law_index', 'section_index', 'para_index', 'text'])

abstracts_df.to_csv('abstracts.csv', index=False)
laws_df.to_csv('laws.csv', index=False)

abstracts_df.head(20)

,abstract_index,para_index,text
0,0,0,Article 1 of the Law establishes the following:
1,0,1,"the anti-terrorist operation period is the time between the date of entry into force of the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014, and the date of entry into force of the Decree of the President of Ukraine on cessation of the anti-terrorist operation or military action on the territory of Ukraine;\nthe anti-terrorist operation territory is the territory of Ukraine containing population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was initiated according to the Decree of the President of Ukraine ""On the Resolution of the Council for National Security and Defense of Ukraine of April 13, 2014 ""On the Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine"" of April 14, 2014 N405/2014 was being carried out."
2,0,2,"During the anti-terrorist operation period, it is forbidden to accrue fines and/or penalties on the principal amounts of liabilities under credit agreements and loan agreements concluded after April 14, 2014 by citizens of Ukraine who are registered and permanently residing, or, after April 14, 2014, relocated from population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, and to legal entities and natural persons – entrepreneurs who conduct (conducted) their business activity on the territory of population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out (Article 2 of the Law)."
3,0,3,"According to Article 3 of the Law, bodies and officials that are authorized by law to carry out state supervision (oversight) in the sphere of commercial activity, during the anti-terrorist operation period and on the anti-terrorist operation territory are temporarily forbidden to hold scheduled and non-scheduled inspections of business entities that operate in the anti-terrorist operation area, except for unscheduled inspections of business entities that are classified as business entities with a high risk level, according to the business risk level assessment criteria approved by the Cabinet of Ministers of Ukraine."
4,0,4,Specifics of state registration of legal entities and natural persons – entrepreneurs from the anti-terrorist operation territory during the anti-terrorist operation period are determined by Article 4 of the Law.
5,0,5,Licenses and permits issued to business entities that operate on the anti-terrorist operation territory which have expired during the anti-terrorist operation period are considered to be extended for the duration of the anti-terrorist operation period (Article 5 of the Law).
6,0,6,"According to Article 6 of the Law, during the anti-terrorist operation period, business entities that operate on the anti-terrorist operation territory are exempt from paying for the use of state- and municipal owned land plots."
7,0,7,"Article 7 of the Law suspends rent payment for the use of state- and municipal owned property by business entities that operate on the anti-terrorist operation territory, for the duration of the anti-terrorist operation period."
8,0,8,"A moratorium on accrual of fines for late payments for municipal and utilities services, accrued for citizens of Ukraine residing in population centers determined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was carried out, is established by Article 8 of the Law."
9,0,9,"According to Article 9 of the Law, during the effect of this Law, mortgaged immovable property located on the anti-terrorist operation territory and 

In [91]:
laws_df.head(20)

,law_index,section_index,para_index,text
0,0,0,0,Article 1. Definitions
1,0,0,1,The period of the anti-terrorist operation is the time between the date of entry into force of the Decree of the President of Ukraine “On the Decision of the National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014 and the date of enactment of the Decree of the President of Ukraine on the completion of the anti-terrorist operation or military actions within the territory of Ukraine.
2,0,0,2,"The territory of the anti-terrorist operation in the territory of Ukraine, where settlements defined in the list approved by the Cabinet of Ministers of Ukraine are located and where the anti-terrorist operation was conducted, which was initiated under the Decree of the President of Ukraine “On the Decision of National Security and Defense Council of Ukraine of 13 April 2014 “On Urgent Measures to Overcome the Terrorist Threat and Preserve the Territorial Integrity of Ukraine” dated 14 April 2014 No. 405/2014."
3,0,1,0,Article 2. The moratorium on the fulfilment of contractual obligations and the accrual of penalties and fines on the principal amount of debt under the credit and other contractual obligations
4,0,1,1,"For the duration of the anti-terrorist operation, it is prohibited to charge the citizens of Ukraine who are registered and permanently reside in or resettled in the period starting from 14 April 2014 from localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted, penalties and/or fines on the principal amount of debt under credit agreements and loan agreements dated from 14 April 2014. The same shall apply to legal entities and individual entrepreneurs who conduct (conducted) their economic activities within the territory of localities defined in the list approved by the Cabinet of Ministers of Ukraine where the anti-terrorist operation was conducted."
5,0,1,2,"Banks and other financial institutions, as well as creditors, are obliged to cancel the penalties and/or fines accrued on the principal amount of debt under credit agreements and loan agreements during the period of the anti-terrorist operation for persons specified in this Article."
6,0,1,3,"Provisions of parts one and two of this Article shall not apply to the accrual of penalties and/or fines on the principal amount of debt under credit agreements and loan agreements concluded with legal entities and individual entrepreneurs located within the territory of anti-terrorist operation, except for localities from the lists provided for in part four of Article 4 of this Law, if such agreements are concluded after 1 January 2018 or to which, by agreement of the parties, changes were made after 1 January 2018 to extend the term for the fulfilment of obligations and/or reduce the amount of interest or punitive sanctions."
7,0,1,4,"For temporarily displaced higher education institutions and temporarily displaced scientific institutions to fulfil credit and other contractual obligations (except for agreements on the provision of educational services) that arose before moving to the territory controlled by the Ukrainian authorities in connection with the anti-terrorist operation, a moratorium is introduced for the period of the anti-terrorist operation."
8,0,1,5,"For the duration of the anti-terrorist operation, it is prohibited to charge penalties and/or fines on the principal amount of debt owed by temporarily displaced higher education institutions and temporarily displaced scientific institutions under credit and other agreements (except for agreements on the provision of educational services) that have not ceased, starting from the date of publication of the order to change their location by the central executive authority in the field of education and science."
9,0,1,6,"A temporari